In [16]:
import pandas as pd

df = pd.read_csv("train_cleaned_data.csv")
df.head()

FileNotFoundError: File b'train_cleaned_data.csv' does not exist

In [17]:
remove_stopwords = []
for index, row in df.iterrows():
    s = str(row['string_remove_stopwords'])
    s = s.split()
    remove_stopwords.append(s)

df["remove_stopwords"] = remove_stopwords
# df.head()

In [18]:
lst_keywords_combined = []
for index, row in df.iterrows():
    s = str(row['keywords_combined'])
    s = s.split()
    lst_keywords_combined.append(s)

df["lst_keywords_combined"] = lst_keywords_combined
df.head()

,id,string_remove_stopwords,company_lower,occupation_lower,place_lower,keywords_combined,remove_stopwords,lst_keywords_combined
0,2502,would create closing game plan would brought d...,uniqlo,sales floor supervisor and visuals merchandiser,5 av and 53 st and times square,would create closing game plan would brought d...,"[would, create, closing, game, plan, would, br...","[would, create, closing, game, plan, would, br..."
1,1605,every day learning day dealing every different...,uniqlo,sale advisor,"london, eng",every day learning day dealing every different...,"[every, day, learning, day, dealing, every, di...","[every, day, learning, day, dealing, every, di..."
2,2438,work environment conducive worklife balance ac...,singtel,sales & accounts operations executive,singapore,work environment conducive worklife balance ac...,"[work, environment, conducive, worklife, balan...","[work, environment, conducive, worklife, balan..."
3,818,work branch theres much information provide ti...,ocbc bank,personal financial consultant,seremban,work branch theres much information provide ti...,"[work, branch, theres, much, information, prov...","[work, branch, theres, much, information, prov..."
4,1079,open culture openminded bosses supportive coll...,starhub,senior hr partner,uni,open culture openminded bosses supportive coll...,"[open, culture, openminded, bosses, supportive...","[open, culture, openminded, bosses, supportive..."


In [19]:
def tf_(doc):
    frequencies = {} 
#     print("doc", type(doc))
    for term in doc:
#         print("term", term)
        term_count = frequencies.get(term, 0) + 1
        frequencies[term] = term_count 
    return frequencies

# for doc in df['remove_stopwords']:
#     print(tf_(doc))

In [20]:
def df_(docs):
    df = {}
    for doc in docs:
        for term in set(doc):
            df_count = df.get(term, 0) + 1 
            df[term] = df_count
    return df

# print(df_(df['remove_stopwords']))

In [21]:
def idf_(df, corpus_size):
    import math
    idf = {}
    for term, freq in df.items():
        idf[term] = round(math.log((corpus_size) / (freq)),2) 
    return idf

# print(idf_(df_(df['remove_stopwords']), len(df['remove_stopwords'])))

In [22]:
def _score(query, doc, docs, k1=1.5, b=0.75): 
    score = 0.0
    tf = tf_(doc)
    df = df_(docs)
    idf = idf_(df, len(docs))
    avg_doc_len = sum(len(doc) for doc in docs)/len(docs) 
    for term in query:
        if term not in tf.keys(): 
            continue
        #
        numerator = idf[term] * tf[term] * (k1 + 1)
        denominator = tf[term] + k1 * (1 - b + b * len(doc) / avg_doc_len) 
        score += (numerator / denominator)
    #
#     print(score)
    return score

In [23]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

stop_words = set(stopwords.words('english'))

query_input = input("Enter query: ").lower()
query_no_punctuation = str(query_input).translate(str.maketrans('', '', string.punctuation))
query = query_no_punctuation.split()
# query = [w for w in query_split if not w.lower() in stop_words]

print(query_input)
# print("query_split: ", query_split)
# print("query_no_punctuation: ", query_no_punctuation)
print("query: ", query)

# Charles One eazy for the

[nltk_data] Downloading package stopwords to /Users/PRASH/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Enter query: is uniqlo a good place to work at?
is uniqlo a good place to work at?
query:  ['is', 'uniqlo', 'a', 'good', 'place', 'to', 'work', 'at']


## To compare between reviews and reviews + occupation + location + company

In [24]:
doc = df['remove_stopwords'][0]
print("doc: ", doc)

docs = df['remove_stopwords']

_score(query, doc, docs, k1=1.5, b=0.75)

doc:  ['would', 'create', 'closing', 'game', 'plan', 'would', 'brought', 'date', 'soon', 'clocked', 'making', 'sure', 'full', 'sky', 'floor', 'well', 'walking', 'sf', 'making', 'sure', 'proper', 'color', 'flow', 'making', 'sure', 'room', 'makes', 'sense']


0.0

In [25]:
doc = df['lst_keywords_combined'][0]
print("doc: ", doc)

docs = df['lst_keywords_combined']

_score(query, doc, docs, k1=1.5, b=0.75)

doc:  ['would', 'create', 'closing', 'game', 'plan', 'would', 'brought', 'date', 'soon', 'clocked', 'making', 'sure', 'full', 'sky', 'floor', 'well', 'walking', 'sf', 'making', 'sure', 'proper', 'color', 'flow', 'making', 'sure', 'room', 'makes', 'sense', 'sales', 'floor', 'supervisor', 'and', 'visuals', 'merchandiser', 'uniqlo', '5', 'av', 'and', '53', 'st', 'and', 'times', 'square']


0.692611668431348

## To find max

In [26]:
import time
docs = df['lst_keywords_combined']
score = []

start_time = time.time()
for i in range(len(docs)):
    doc = docs[i]
#     print(doc)
    bm25 = _score(query, doc, docs, k1=1.5, b=0.75)
    score.append(bm25)
end_time = time.time()

In [27]:
m = max(score)
time_taken = end_time - start_time
idx = score.index(max(score))

print("Max Score: ", m)
print(f"Time taken: {time_taken}s")
print('index ' + str(idx) + ': ' + df['company_lower'][idx] + ', ' + df['occupation_lower'][idx] + ', ' + df['place_lower'][idx])

Max Score:  10.393648548755886
Time taken: 244.4947850704193s
index 680: singtel, to handle project team members by assisting, serangoon north


## How about top 5 results instead

In [33]:
score_dict = {k:v for k, v in enumerate(score)}
# print(score_dict)

sorted_score_dict = {k: v for k, v in sorted(score_dict.items(), key=lambda item: item[1], reverse = True)}
# print(sorted_score_dict)


rank = 0
for k,v in sorted_score_dict.items():
    rank = rank + 1
    print("\n")
    print(f"Rank Number --> {rank}")
    print(f"Document is --> {df['lst_keywords_combined'][k]}")
    print(f"Cosine Similarity score is --> {v}")
    print("\n")
    
    #top ten
    if rank == 10:
        break




Rank Number --> 1
Document is --> ['handle', 'project', 'team', 'members', 'assisting', 'paper', 'work', 'filing', 'taking', 'minutes', 'meeting', 'work', 'site', 'office', 'to', 'handle', 'project', 'team', 'members', 'by', 'assisting', 'singtel', 'serangoon', 'north']
Cosine Similarity score is --> 10.393648548755886




Rank Number --> 2
Document is --> ['friendly', 'cooperative', 'colleagues', 'good', 'teamwork', 'challenging', 'tasks', 'tight', 'datelines', 'end', 'month', 'supportive', 'understanding', 'boss', 'flexible', 'lunch', 'hours', 'assist', 'at', 'team', 'bonding', 'singtel', 'pasir', 'ris', 'exchange', 'and', 'jurong', 'east', 'exchange']
Cosine Similarity score is --> 9.84464589479124




Rank Number --> 3
Document is --> ['routine', 'day', 'day', 'work', 'monitoring', 'troubleshooting', 'issues', 'projects', 'run', 'specific', 'date', 'lines', 'working', 'good', 'pool', 'cooperative', 'team', 'good', 'team', 'work', 'challenging', 'task', 'meet', 'user', 'expectatio

In [ ]:
#Relevant Documents

#BM25

#Query 1: is uniqlo a good place to work at?

# Model 1:  5,6,7,8,9
# Model 2:  3,6,7,8,9



#Query 2: does dbs pay its employees well?

#Model 1: 1,3,5,8
#Model 2:  2,5,9



#Query 3: Does singtel have a nice working environment?

#Model 1: 2,3,5,6,7,8,9,10
#Model 2: nil



#Query 4: What kind of Jobs are there in kuala lumpur?

#Model 1: 1,2,3,4,5,7,8,9,10
#Model 2: nil



#Query 5: Best place to work as a manager

#Model 1: 3,5,8,10
#Model 2: nil

In [29]:
top_results = pd.DataFrame(sorted_score_dict.items(), columns=['Index', 'bm25_results'])

# print(top_results['Index'])

result_company = []
result_occupation = []
result_place = []
result_keywords = []

for i in range(len(top_results['Index'])):
    result_company.append(df['company_lower'][top_results['Index'][i]])
    result_occupation.append(df['occupation_lower'][top_results['Index'][i]])
    result_place.append(df['place_lower'][top_results['Index'][i]])
    result_keywords.append(df['keywords_combined'][top_results['Index'][i]])
    
top_results['Company'] = result_company
top_results['Occupation'] = result_occupation
top_results['Place'] = result_place
top_results['Keywords'] = result_keywords

top_results.head(10)

ValueError: DataFrame constructor not properly called!

In [35]:
#MEAN RECIPROCAL RANK  (MODEL 1)
import numpy as np

def model_1_mean_reciprocal_rank(rs):
    """Score is reciprocal of the rank of the first relevant item

    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).

    Example from http://en.wikipedia.org/wiki/Mean_reciprocal_rank
    >>> rs = [[0, 0, 1], [0, 1, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.61111111111111105
    >>> rs = np.array([[0, 0, 0], [0, 1, 0], [1, 0, 0]])
    >>> mean_reciprocal_rank(rs)
    0.5
    >>> rs = [[0, 0, 0, 1], [1, 0, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.75

    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Mean reciprocal rank
    """
#     query_1_score = [0,0,0,0,1,1,1,1,1,0]
#     query_2_score = [1,0,1,0,1,0,0,1,0,0]
#     query_3_score = [0,1,1,0,1,1,1,1,1,1]
#     query_4_score = [1,1,1,1,1,0,1,1,1,1]
#     query_5_score = [0,0,1,0,1,0,0,1,0,1]
   
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])

rs = [[0,0,0,0,1,1,1,1,1,0],[1,0,1,0,1,0,0,1,0,0],[0,1,1,0,1,1,1,1,1,1],[1,1,1,1,1,0,1,1,1,1],[0,0,1,0,1,0,0,1,0,1]]
model_1_mean_reciprocal_rank(rs)


0.6066666666666667

In [36]:
#MEAN RECIPROCAL RANK  (MODEL 2)
import numpy

def model_2_mean_reciprocal_rank(rs):
    """Score is reciprocal of the rank of the first relevant item

    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).

    Example from http://en.wikipedia.org/wiki/Mean_reciprocal_rank
    >>> rs = [[0, 0, 1], [0, 1, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.61111111111111105
    >>> rs = np.array([[0, 0, 0], [0, 1, 0], [1, 0, 0]])
    >>> mean_reciprocal_rank(rs)
    0.5
    >>> rs = [[0, 0, 0, 1], [1, 0, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.75

    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Mean reciprocal rank
    """
#     query_1_score = [0,0,1,0,0,1,1,1,1,0]
#     query_2_score = [0,1,0,0,1,0,0,0,1,0]
#     query_3_score = [0,0,0,0,0,0,0,0,0,0]
#     query_4_score = [0,0,0,0,0,0,0,0,0,0]
#     query_5_score = [0,0,0,0,0,0,0,0,0,0]
   
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])

rs = [[0,0,1,0,0,1,1,1,1,0],[0,1,0,0,1,0,0,0,1,0],[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0]]
model_2_mean_reciprocal_rank(rs)


0.16666666666666666

In [ ]:
#MODEL 1 IS BETTER, SCORE IS CLOSER TO 1 --> FIRST SEARCH RESULT (DOCUMENT) MORE LIKELY TO BE RELEVANT THAN THAT OF MODEL 2

In [ ]:
#AVERAGE PRECISION AND MEAN AVERAGE PRECISION (MODEL 1)

In [37]:
def precision_at_k(r, k):
    """Score is precision @ k

    Relevance is binary (nonzero is relevant).

    >>> r = [0, 0, 1]
    >>> precision_at_k(r, 1)
    0.0
    >>> precision_at_k(r, 2)
    0.0
    >>> precision_at_k(r, 3)
    0.33333333333333331
    >>> precision_at_k(r, 4)
    Traceback (most recent call last):
        File "<stdin>", line 1, in ?
    ValueError: Relevance score length < k


    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Precision @ k

    Raises:
        ValueError: len(r) must be >= k
    """
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)



In [38]:
def average_precision(r):
    """Score is average precision (area under PR curve)

    Relevance is binary (nonzero is relevant).

    >>> r = [1, 1, 0, 1, 0, 1, 0, 0, 0, 1]
    >>> delta_r = 1. / sum(r)
    >>> sum([sum(r[:x + 1]) / (x + 1.) * delta_r for x, y in enumerate(r) if y])
    0.7833333333333333
    >>> average_precision(r)
    0.78333333333333333

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Average precision
    """
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

In [39]:
def model1_average_precision_query1(r):
    """Score is average precision (area under PR curve)

    Relevance is binary (nonzero is relevant).

    >>> r = [1, 1, 0, 1, 0, 1, 0, 0, 0, 1]
    >>> delta_r = 1. / sum(r)
    >>> sum([sum(r[:x + 1]) / (x + 1.) * delta_r for x, y in enumerate(r) if y])
    0.7833333333333333
    >>> average_precision(r)
    0.78333333333333333

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Average precision
    """
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)


model1_average_precision_query1([0,0,0,0,1,1,1,1,1,0])

0.40349206349206346

In [40]:
def model1_average_precision_query2(r):
    """Score is average precision (area under PR curve)

    Relevance is binary (nonzero is relevant).

    >>> r = [1, 1, 0, 1, 0, 1, 0, 0, 0, 1]
    >>> delta_r = 1. / sum(r)
    >>> sum([sum(r[:x + 1]) / (x + 1.) * delta_r for x, y in enumerate(r) if y])
    0.7833333333333333
    >>> average_precision(r)
    0.78333333333333333

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Average precision
    """
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

model1_average_precision_query2([1,0,1,0,1,0,0,1,0,0])

0.6916666666666667

In [41]:
def model1_average_precision_query3(r):
    """Score is average precision (area under PR curve)

    Relevance is binary (nonzero is relevant).

    >>> r = [1, 1, 0, 1, 0, 1, 0, 0, 0, 1]
    >>> delta_r = 1. / sum(r)
    >>> sum([sum(r[:x + 1]) / (x + 1.) * delta_r for x, y in enumerate(r) if y])
    0.7833333333333333
    >>> average_precision(r)
    0.78333333333333333

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Average precision
    """
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

model1_average_precision_query3([0,1,1,0,1,1,1,1,1,1])

0.6844246031746032

In [42]:
def model1_average_precision_query4(r):
    """Score is average precision (area under PR curve)

    Relevance is binary (nonzero is relevant).

    >>> r = [1, 1, 0, 1, 0, 1, 0, 0, 0, 1]
    >>> delta_r = 1. / sum(r)
    >>> sum([sum(r[:x + 1]) / (x + 1.) * delta_r for x, y in enumerate(r) if y])
    0.7833333333333333
    >>> average_precision(r)
    0.78333333333333333

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Average precision
    """
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

model1_average_precision_query4([1,1,1,1,1,0,1,1,1,1])

0.9467813051146384

In [43]:
def model1_average_precision_query5(r):
    """Score is average precision (area under PR curve)

    Relevance is binary (nonzero is relevant).

    >>> r = [1, 1, 0, 1, 0, 1, 0, 0, 0, 1]
    >>> delta_r = 1. / sum(r)
    >>> sum([sum(r[:x + 1]) / (x + 1.) * delta_r for x, y in enumerate(r) if y])
    0.7833333333333333
    >>> average_precision(r)
    0.78333333333333333

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Average precision
    """
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

model1_average_precision_query5([0,0,1,0,1,0,0,1,0,1])

0.3770833333333333

In [44]:
def model1_mean_average_precision(rs):
    """Score is mean average precision

    Relevance is binary (nonzero is relevant).

    >>> rs = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1]]
    >>> mean_average_precision(rs)
    0.78333333333333333
    >>> rs = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1], [0]]
    >>> mean_average_precision(rs)
    0.39166666666666666

    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Mean average precision
    """
    return np.mean([average_precision(r) for r in rs])

rs = [[0,0,0,0,1,1,1,1,1,0],[1,0,1,0,1,0,0,1,0,0],[0,1,1,0,1,1,1,1,1,1],[1,1,1,1,1,0,1,1,1,1],[0,0,1,0,1,0,0,1,0,1]]
model1_mean_average_precision(rs)

0.6206895943562609

In [ ]:
#AVERAGE PRECISION AND MEAN AVERAGE PRECISION (MODEL 2)

In [51]:
def model2_average_precision_query1(r):
    """Score is average precision (area under PR curve)

    Relevance is binary (nonzero is relevant).

    >>> r = [1, 1, 0, 1, 0, 1, 0, 0, 0, 1]
    >>> delta_r = 1. / sum(r)
    >>> sum([sum(r[:x + 1]) / (x + 1.) * delta_r for x, y in enumerate(r) if y])
    0.7833333333333333
    >>> average_precision(r)
    0.78333333333333333

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Average precision
    """
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)


model2_average_precision_query1([0,0,1,0,0,1,1,1,1,0])

0.43015873015873013

In [52]:
def model2_average_precision_query2(r):
    """Score is average precision (area under PR curve)

    Relevance is binary (nonzero is relevant).

    >>> r = [1, 1, 0, 1, 0, 1, 0, 0, 0, 1]
    >>> delta_r = 1. / sum(r)
    >>> sum([sum(r[:x + 1]) / (x + 1.) * delta_r for x, y in enumerate(r) if y])
    0.7833333333333333
    >>> average_precision(r)
    0.78333333333333333

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Average precision
    """
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

model2_average_precision_query2([0,1,0,0,1,0,0,0,1,0])

0.41111111111111115

In [53]:
def model2_average_precision_query3(r):
    """Score is average precision (area under PR curve)

    Relevance is binary (nonzero is relevant).

    >>> r = [1, 1, 0, 1, 0, 1, 0, 0, 0, 1]
    >>> delta_r = 1. / sum(r)
    >>> sum([sum(r[:x + 1]) / (x + 1.) * delta_r for x, y in enumerate(r) if y])
    0.7833333333333333
    >>> average_precision(r)
    0.78333333333333333

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Average precision
    """
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

model2_average_precision_query3([0,0,0,0,0,0,0,0,0,0])

0.0

In [54]:
def model2_average_precision_query4(r):
    """Score is average precision (area under PR curve)

    Relevance is binary (nonzero is relevant).

    >>> r = [1, 1, 0, 1, 0, 1, 0, 0, 0, 1]
    >>> delta_r = 1. / sum(r)
    >>> sum([sum(r[:x + 1]) / (x + 1.) * delta_r for x, y in enumerate(r) if y])
    0.7833333333333333
    >>> average_precision(r)
    0.78333333333333333

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Average precision
    """
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

model2_average_precision_query4([0,0,0,0,0,0,0,0,0,0])

0.0

In [55]:
def model2_average_precision_query5(r):
    """Score is average precision (area under PR curve)

    Relevance is binary (nonzero is relevant).

    >>> r = [1, 1, 0, 1, 0, 1, 0, 0, 0, 1]
    >>> delta_r = 1. / sum(r)
    >>> sum([sum(r[:x + 1]) / (x + 1.) * delta_r for x, y in enumerate(r) if y])
    0.7833333333333333
    >>> average_precision(r)
    0.78333333333333333

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Average precision
    """
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

model2_average_precision_query5([0,0,0,0,0,0,0,0,0,0])

0.0

In [56]:
def model2_mean_average_precision(rs):
    """Score is mean average precision

    Relevance is binary (nonzero is relevant).

    >>> rs = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1]]
    >>> mean_average_precision(rs)
    0.78333333333333333
    >>> rs = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1], [0]]
    >>> mean_average_precision(rs)
    0.39166666666666666

    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Mean average precision
    """
    return np.mean([average_precision(r) for r in rs])

rs = [[0,0,1,0,0,1,1,1,1,0],[0,1,0,0,1,0,0,0,1,0],[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0]]
model2_mean_average_precision(rs)

0.16825396825396827